# Les traits

---

> A ce point du cours, vous avez déjà entendu parler de traits quelques dizaines de fois et la définition de ce qu'est un trait vous est déjà normalement familière. Pour résumer, **un trait offre des méthodes et des propriétés aux types**. et il y en a une myriade déjà présents dans la librairie standard ainsi que des milliers d'implémentations de ces derniers pour tous les types confondus.

> Nous avons aussi la possibilité **d'implementer les traits de la std pour nos propres types** et aussi la possibilité de **creer nos propres traits**.

> La création de trait sera détaillée dans le chapitre 12, pour le moment, je vous invite à comprendre l'implémentation du trait From et de trait Into pour un de nos propres types, c'est en général les premiers trait qu'un nouveau développeur Rust doit implémenter (Le prochain est souvent le trait Iterator).

## Le trait From

### Documentation

- La documentation de Rust donne cette définition pour le trait From :

```
Trait std::convert::From

pub trait From<T>: Sized {
    // Required method
    fn from(value: T) -> Self;
}
```

**Used to do value-to-value conversions while consuming the input value. It is the reciprocal of Into.**

```
Generic Implementations

    From<T> for U implies Into<U> for T
    From is reflexive, which means that From<T> for T is implemented

```
> Le trait From permet donc la conversion d'un type `T` en un autre `Self`. Et implémenter From pour un type offre au développeur l'implémentation gratuite de son contraire, le trait Into.

### Exemple d'implémentation de la std

> Regardons par exemple l'implémentation de From<&str> for String de la std :
```
impl From<&str> for String {
    /// Converts a `&str` into a [`String`].
    ///
    /// The result is allocated on the heap.
    #[inline]
    fn from(s: &str) -> String {
        s.to_owned()
    }
}
```
- On utilisera cette implémentation ainsi :

In [ ]:
let s: String = From::<&str>::from("Bananes volantes"); // forme non inferee
let s: String = From::from("Bananes volantes"); // forme inferee

let s = Into::<String>::into("Bananes volantes"); // Implementation reciproque via Into
let s: String = "Bananes volantes".into(); // Implementation inferee de Into

## Implémentation de From pour une énumération

- Imaginons une énumération avec les trois variants suivant :

In [7]:
{
    #[derive(Debug, Copy, Clone)]
    enum RelativeStatus {
        Positive,
        Null,
        Negative,
    }
}

()

> Le but de l'implémentation que nous allons faire, c'est de classer les nombres entiers relatifs en 3 catégories, supérieur a zéro, nul ou inférieur a 0.

> Nous allons donc faire une implémentation de `From<isize> for RelativeStatus`.

- Le mieux pour commencer est d'écrire une fonction pour se faire une idée :

In [11]:
{
    #[derive(Debug, Copy, Clone)]
    enum RelativeStatus {
        Positive,
        Null,
        Negative,
    }
    fn get_relative_status(r: isize) -> RelativeStatus {
        if r > 0 {
            RelativeStatus::Positive
        } else if r < 0 {
            RelativeStatus::Negative
        } else {
            RelativeStatus::Null
        }
    }
    dbg!(get_relative_status(0));
    dbg!(get_relative_status(7));
    dbg!(get_relative_status(-7));
}

[src/lib.rs:54] get_relative_status(0) = Null
[src/lib.rs:55] get_relative_status(7) = Positive
[src/lib.rs:56] get_relative_status(-7) = Negative


()

> Le code de cette fonction ne devrait poser aucun problème.

- Intégrons maintenant le corps de cette fonction dans une implémentation de From :

In [9]:
{
    #[derive(Debug, Copy, Clone)]
    enum RelativeStatus {
        Positive,
        Null,
        Negative,
    }
    impl From<isize> for RelativeStatus {
        fn from(r: isize) -> Self {
            if r > 0 {
                RelativeStatus::Positive
            } else if r < 0 {
                RelativeStatus::Negative
            } else {
                RelativeStatus::Null
            }
        }
    }
    dbg!(Into::<RelativeStatus>::into(0)); // Je profite de l'implementation gratuite du trait Into !
    dbg!(Into::<RelativeStatus>::into(7));
    dbg!(Into::<RelativeStatus>::into(-7));
}

[src/lib.rs:56] Into::<RelativeStatus>::into(0) = Null
[src/lib.rs:57] Into::<RelativeStatus>::into(7) = Positive
[src/lib.rs:58] Into::<RelativeStatus>::into(-7) = Negative


()

> **Pour implémenter les traits de la std, on remplace les types génériques par nos types et on suit la documentation. Il n'y a pas de réelle difficulté.**

## Exercices

### Reprenez l'exemple précèdent et implémentez From\<RelativeStatus> for  isize :

- disons que le résultat sera le isize minimal pour Negative, 0 pour Null, et le isize maximal pour Positive :

In [ ]:
{
    #[derive(Debug, Copy, Clone)]
    enum RelativeStatus {
        Positive,
        Null,
        Negative,
    }

    // IMPLEMENTATION DU FROM ICI

    dbg!(Into::<isize>::into(RelativeStatus::Null));
    dbg!(Into::<isize>::into(RelativeStatus::Positive));
    dbg!(Into::<isize>::into(RelativeStatus::Negative));
}

In [15]:
 {
    #[derive(Debug, Copy, Clone)]
    enum RelativeStatus {
        Positive,
        Null,
        Negative,
    }
    impl From<RelativeStatus> for isize {
        fn from(r: RelativeStatus) -> Self {
            match r {
                RelativeStatus::Positive => isize::MAX,
                RelativeStatus::Negative => isize::MIN,
                RelativeStatus::Null => 0,
            }
        }
    }    
    dbg!(Into::<isize>::into(RelativeStatus::Null));
    dbg!(Into::<isize>::into(RelativeStatus::Positive));
    dbg!(Into::<isize>::into(RelativeStatus::Negative));
}

[src/lib.rs:54] Into::<isize>::into(RelativeStatus::Null) = 0
[src/lib.rs:55] Into::<isize>::into(RelativeStatus::Positive) = 9223372036854775807
[src/lib.rs:56] Into::<isize>::into(RelativeStatus::Negative) = -9223372036854775808


()